# Comparative Analysis of ONA, CMA, and DEMA Algorithms for Aethalometer Data

## 1. Introduction and Background

This notebook implements and compares three post-processing algorithms for micro-aethalometer data:
  
1. **Optimized Noise-reduction Algorithm (ONA)**: A method described by Hagler et al. (2011) that adaptively time-averages BC data based on incremental light attenuation (ΔATN).

2. **Centered Moving Average (CMA)**: A smoothing technique that incorporates data points both before and after each measurement to reduce noise while preserving microenvironmental characteristics.

3. **Double Exponentially Weighted Moving Average (DEMA)**: A smoothing approach that reduces noise-induced artifacts while limiting lag, especially useful for source apportionment calculations.
 
Both CMA and DEMA have been shown to outperform ONA for newer dual-spot aethalometers in recent research by Liu et al. (2021) and Mendoza et al. (2024).


## 2. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
import seaborn as sns
from IPython.display import display

# Set plot style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## 3. Load and Explore the Data
 
First, let's load the Aethalometer data and examine its structure.

In [ ]:
# Define the file path - replace with your actual file path
file_path = "your_aethalometer_data.csv"  # Replace with your file path

# Load the data
data = pd.read_csv(file_path)

# Display basic information about the dataset
print(f"Dataset shape: {data.shape}")
print("\nColumn names:")
print(data.columns.tolist())

# Display the first few rows
print("\nFirst few rows of the dataset:")
display(data.head())

# Check for the presence of BC columns for each wavelength
wavelengths = ['UV', 'Blue', 'Green', 'Red', 'IR']
for wavelength in wavelengths:
    bc_col = f"{wavelength} BC1"
    atn_col = f"{wavelength} ATN1"
    
    if bc_col in data.columns:
        print(f"\n{wavelength} wavelength data:")
        print(f"  BC range: {data[bc_col].min()} to {data[bc_col].max()} ng/m³")
        print(f"  Negative BC values: {(data[bc_col] < 0).sum()} ({(data[bc_col] < 0).sum() / len(data) * 100:.2f}%)")
        
        if atn_col in data.columns:
            print(f"  ATN range: {data[atn_col].min()} to {data[atn_col].max()}")
    else:
        print(f"\nWarning: {wavelength} data columns not found")

# Check the time resolution
if 'Timebase (s)' in data.columns:
    timebase = data['Timebase (s)'].iloc[0]
    print(f"\nInstrument timebase: {timebase} seconds")
else:
    print("\nTimebase column not found")

## 4. ONA Algorithm Implementation
  
The Optimized Noise-reduction Algorithm (ONA) adaptively time-averages BC data based on incremental light attenuation (ΔATN).

In [ ]:
def apply_ona(data, wavelength='Blue', delta_atn_min=0.05):
    """
    Apply the Optimized Noise-reduction Algorithm to Aethalometer data
    
    Parameters:
    -----------
    data : pandas.DataFrame
        DataFrame containing Aethalometer data with columns for timestamp, BC, and ATN values
    wavelength : str
        Which wavelength to process ('UV', 'Blue', 'Green', 'Red', 'IR')
    delta_atn_min : float
        Minimum change in attenuation (ATN) required for averaging (default 0.05)
        
    Returns:
    --------
    data_smoothed : pandas.DataFrame
        DataFrame with the original data plus additional columns for smoothed BC and number of points averaged
    """
    # Create a copy of the input dataframe
    data_smoothed = data.copy()
    
    # Add a column for number of points averaged (initially set to 1)
    points_averaged_col = f"{wavelength}_points_averaged"
    data_smoothed[points_averaged_col] = 1
    
    # Identify the columns for BC and ATN values based on wavelength
    bc_col = f"{wavelength} BC1"  # BC1 column for this wavelength
    atn_col = f"{wavelength} ATN1"  # ATN1 column for this wavelength
    
    # Add a column for ONA smoothed BC (initially just a copy)
    smoothed_bc_col = f"{wavelength}_BC_ONA"
    data_smoothed[smoothed_bc_col] = data_smoothed[bc_col].copy()
    
    # Check if ATN column exists
    if atn_col not in data_smoothed.columns:
        print(f"Warning: {atn_col} column not found. ONA algorithm cannot be applied.")
        return data_smoothed
    
    # Identify filter changes (large jumps in ATN)
    temp = np.zeros(len(data_smoothed))
    for i in range(1, len(data_smoothed)-1):
        temp[i] = abs(data_smoothed[atn_col].iloc[i+1] - data_smoothed[atn_col].iloc[i])
    
    # Find points where ATN changed drastically (filter changes)
    # Using threshold of 30 as in the paper
    filter_changes = np.where(np.logical_or(temp > 30, np.isnan(temp)))[0]
    
    # Create filtchange array with start and end points
    if len(filter_changes) > 0:
        filtchange = np.zeros(len(filter_changes) + 2, dtype=int)
        filtchange[1:-1] = filter_changes
        filtchange[-1] = len(data_smoothed)
    else:
        filtchange = np.array([0, len(data_smoothed)], dtype=int)
    
    print(f"Number of filter changes detected for {wavelength}: {len(filter_changes)}")
    
    # Process each segment between filter changes
    for k in range(len(filtchange) - 1):
        j = filtchange[k] + 1  # Set to first point after filter change
        
        while j < filtchange[k+1]:
            # Current ATN value
            current_atn = data_smoothed[atn_col].iloc[j]
            
            # Find points where ATN increases by at most delta_atn_min
            end_idx = filtchange[k+1]
            search_range = data_smoothed[atn_col].iloc[j+1:end_idx]
            
            if len(search_range) > 0:
                # Find points where ATN <= current_atn + delta_atn_min
                des_ind = np.where(search_range <= current_atn + delta_atn_min)[0]
                
                if len(des_ind) > 0:
                    # Calculate range of points to average
                    end_j = min(j + des_ind[-1] + 1, len(data_smoothed))
                    
                    # Check that j < end_j and there is data to average
                    if j < end_j:
                        # Get the data slice
                        bc_slice = data_smoothed[bc_col].iloc[j:end_j]
                        
                        # Check if there is valid data to average
                        if len(bc_slice) > 0 and not bc_slice.isna().all():
                            # Calculate smoothed BC by averaging over the window
                            avg_bc = np.nanmean(bc_slice)
                            
                            # Apply the averaged BC value to all points in the window - using loc
                            row_indices = data_smoothed.index[j:end_j]
                            data_smoothed.loc[row_indices, smoothed_bc_col] = avg_bc
                            
                            # Record number of points used in averaging - using loc
                            data_smoothed.loc[row_indices, points_averaged_col] = end_j - j
                        
                    # Move j to next position after the current window
                    j = end_j
                else:
                    # If no suitable points found, move to next point
                    j += 1
            else:
                # If at the end of the data segment, move to next point
                j += 1
    
    return data_smoothed


## 5. CMA Algorithm Implementation
  
The Centered Moving Average is a smoothing technique that uses data points both before and after each measurement.

In [ ]:
def apply_cma(data, wavelength='Blue', window_size=None):
    """
    Apply the Centered Moving Average algorithm to Aethalometer data
    
    Parameters:
    -----------
    data : pandas.DataFrame
        DataFrame containing Aethalometer data
    wavelength : str
        Which wavelength to process ('UV', 'Blue', 'Green', 'Red', 'IR')
    window_size : int or None
        Size of the moving average window (must be odd). If None, 
        will use a default based on the data's timebase
        
    Returns:
    --------
    data_smoothed : pandas.DataFrame
        DataFrame with the original data plus additional columns for smoothed BC
    """
    # Create a copy of the input dataframe
    data_smoothed = data.copy()
    
    # Identify the column for BC values based on wavelength
    bc_col = f"{wavelength} BC1"
    
    # Determine window size if not specified
    if window_size is None:
        if 'Timebase (s)' in data.columns:
            timebase = data['Timebase (s)'].iloc[0]
            if timebase == 1:
                window_size = 11  # 11 seconds for 1-second data
            elif timebase == 5:
                window_size = 5   # 25 seconds for 5-second data
            elif timebase == 60:
                window_size = 3   # 3 minutes for 1-minute data
            else:
                window_size = 5   # Default for other timebases
        else:
            window_size = 5       # Default if timebase is unknown
    
    # Make sure window_size is odd
    if window_size % 2 == 0:
        window_size += 1
    
    print(f"Using window size of {window_size} for CMA on {wavelength}")
    
    # Add columns for CMA results
    smoothed_bc_col = f"{wavelength}_BC_CMA"
    data_smoothed[smoothed_bc_col] = data_smoothed[bc_col].rolling(
        window=window_size, center=True, min_periods=1
    ).mean()
    
    return data_smoothed

## 6. DEMA Algorithm Implementation
 
The Double Exponentially Weighted Moving Average applies additional smoothing to an EMA to reduce noise while limiting lag.

In [ ]:
def apply_dema(data, wavelength='Blue', alpha=None):
    """
    Apply the Double Exponentially Weighted Moving Average algorithm to Aethalometer data
    
    Parameters:
    -----------
    data : pandas.DataFrame
        DataFrame containing Aethalometer data
    wavelength : str
        Which wavelength to process ('UV', 'Blue', 'Green', 'Red', 'IR')
    alpha : float
        Smoothing parameter (between 0 and 1)
        For 60s data, 0.125 approximates a 15-minute smoothing window
        
    Returns:
    --------
    data_smoothed : pandas.DataFrame
        DataFrame with the original data plus additional columns for smoothed BC
    """
    # Create a copy of the input dataframe
    data_smoothed = data.copy()
    
    # Identify the column for BC values based on wavelength
    bc_col = f"{wavelength} BC1"
    
    # Set the smoothing parameter based on timebase if not explicitly provided
    if 'Timebase (s)' in data.columns:
        timebase = data['Timebase (s)'].iloc[0]
        if alpha is None:
            # Use formula 2/(N+1) where N is the desired smoothing period
            if timebase == 1:
                # Default to approximate 5-minute window for 1-second data
                N = 300 / timebase
            elif timebase == 5:
                # Default to approximate 5-minute window for 5-second data
                N = 300 / timebase
            elif timebase == 60:
                # Default to approximate 15-minute window for 60-second data
                N = 900 / timebase
            else:
                N = 15  # Default for other timebases
                
            alpha = 2 / (N + 1)
    else:
        # Default alpha if timebase is unknown
        if alpha is None:
            alpha = 0.125
    
    print(f"Using alpha of {alpha:.4f} for DEMA on {wavelength}")
    
    # Calculate EMA
    ema_col = f"{wavelength}_EMA"
    # First EMA
    data_smoothed[ema_col] = data_smoothed[bc_col].ewm(alpha=alpha, adjust=False).mean()
    
    # Calculate DEMA: (2 * EMA) - EMA(EMA)
    dema_col = f"{wavelength}_BC_DEMA"
    ema_of_ema = data_smoothed[ema_col].ewm(alpha=alpha, adjust=False).mean()
    data_smoothed[dema_col] = (2 * data_smoothed[ema_col]) - ema_of_ema
    
    return data_smoothed


## 7. Apply All Processing Methods to Data

In [ ]:
# Create separate dataframes for each method
processed_data_raw = data.copy()
processed_data_ona = data.copy()
processed_data_cma = data.copy()
processed_data_dema = data.copy()

# Set the minimum change in attenuation (ΔATN) for ONA
delta_atn_min = 0.05  # Default value from the Hagler paper

# Process each wavelength with all methods
for wavelength in wavelengths:
    bc_col = f"{wavelength} BC1"
    atn_col = f"{wavelength} ATN1"
    
    if bc_col in processed_data_raw.columns:
        # Apply ONA if ATN column exists
        if atn_col in processed_data_raw.columns:
            print(f"\nApplying ONA to {wavelength} wavelength data...")
            processed_data_ona = apply_ona(processed_data_ona, wavelength, delta_atn_min)
        
        # Apply CMA
        print(f"\nApplying CMA to {wavelength} wavelength data...")
        processed_data_cma = apply_cma(processed_data_cma, wavelength)
        
        # Apply DEMA
        print(f"\nApplying DEMA to {wavelength} wavelength data...")
        processed_data_dema = apply_dema(processed_data_dema, wavelength)

# Apply Source Apportionment for all methods
# This step will be included in a separate section

## 8. Calculate Source Apportionment
 
Following the recommendations from recent research, we'll implement source apportionment calculations using the Aethalometer model for each processing method.

In [ ]:
def calculate_source_apportionment(data, aae_wb=2.0, aae_ff=1.0):
    """
    Calculate source apportionment using the Aethalometer Model
    
    Parameters:
    -----------
    data : pandas.DataFrame
        DataFrame with BC data at blue and IR wavelengths
    aae_wb : float
        Absorption Ångström Exponent for wood burning (default 2.0)
    aae_ff : float
        Absorption Ångström Exponent for fossil fuel (default 1.0)
    
    Returns:
    --------
    data_sa : pandas.DataFrame
        DataFrame with additional source apportionment columns
    """
    # Create a copy of the input dataframe
    data_sa = data.copy()
    
    # Get BC values at blue and IR wavelengths (raw data)
    bc_blue = data_sa['Blue BC1']
    bc_ir = data_sa['IR BC1']
    
    # Calculate absorption coefficients
    # For simplicity, using approximate MACs and Cref values from literature
    mac_blue = 10.12  # m²/g at 470nm
    mac_ir = 7.77     # m²/g at 880nm
    c_ref = 1.3       # Multiple scattering enhancement factor
    
    babs_blue = bc_blue * mac_blue / c_ref
    babs_ir = bc_ir * mac_ir / c_ref
    
    # Calculate the ratio of wavelengths for the Aethalometer model
    wavelength_ratio = 470 / 880
    
    # Calculate absorption coefficients for wood burning and fossil fuel at IR wavelength
    # Using the Aethalometer model equations
    babs_ff_ir = (babs_blue - babs_ir * (wavelength_ratio ** (-aae_wb))) / \
                 ((wavelength_ratio ** (-aae_ff)) - (wavelength_ratio ** (-aae_wb)))
                 
    babs_wb_ir = babs_ir - babs_ff_ir
    
    # Calculate BB% (biomass burning percentage)
    bb_percent = 100 * babs_wb_ir / babs_ir
    
    # Handle infinity and NaN values
    bb_percent = bb_percent.replace([np.inf, -np.inf], np.nan)
    bb_percent = bb_percent.fillna(0)
    
    # Store results
    data_sa['Babs_Blue'] = babs_blue
    data_sa['Babs_IR'] = babs_ir
    data_sa['Babs_FF_IR'] = babs_ff_ir
    data_sa['Babs_WB_IR'] = babs_wb_ir
    data_sa['BB_Percent'] = bb_percent
    
    # Calculate BC from wood burning and fossil fuel
    data_sa['BC_WB'] = bc_ir * bb_percent / 100
    data_sa['BC_FF'] = bc_ir * (100 - bb_percent) / 100
    
    return data_sa

In [ ]:
def apply_dema_to_source_apportionment(data, alpha=0.125):
    """
    Apply DEMA specifically to source apportionment results
    
    Parameters:
    -----------
    data : pandas.DataFrame
        DataFrame with source apportionment data
    alpha : float
        Smoothing parameter
        
    Returns:
    --------
    data_smoothed : pandas.DataFrame
        DataFrame with smoothed source apportionment data
    """
    # Create a copy of the input dataframe
    data_smoothed = data.copy()
    
    # Apply DEMA to BB_Percent
    if 'BB_Percent' in data_smoothed.columns:
        # First EMA
        data_smoothed['BB_Percent_EMA'] = data_smoothed['BB_Percent'].ewm(alpha=alpha, adjust=False).mean()
        
        # Calculate DEMA
        ema_of_ema = data_smoothed['BB_Percent_EMA'].ewm(alpha=alpha, adjust=False).mean()
        data_smoothed['BB_Percent_DEMA'] = (2 * data_smoothed['BB_Percent_EMA']) - ema_of_ema
        
        # Limit BB% to logical range [0, 100]
        data_smoothed['BB_Percent_DEMA'] = data_smoothed['BB_Percent_DEMA'].clip(0, 100)
        
        # Recalculate BC_WB and BC_FF using smoothed BB_Percent
        if 'IR_BC_CMA' in data_smoothed.columns:
            bc_ir = data_smoothed['IR_BC_CMA']
        elif 'IR_BC_DEMA' in data_smoothed.columns:
            bc_ir = data_smoothed['IR_BC_DEMA']
        elif 'IR_BC_ONA' in data_smoothed.columns:
            bc_ir = data_smoothed['IR_BC_ONA']
        else:
            bc_ir = data_smoothed['IR BC1']
            
        data_smoothed['BC_WB_DEMA'] = bc_ir * data_smoothed['BB_Percent_DEMA'] / 100
        data_smoothed['BC_FF_DEMA'] = bc_ir * (100 - data_smoothed['BB_Percent_DEMA']) / 100
    
    return data_smoothed

In [ ]:
# Calculate source apportionment for raw data
print("\nCalculating source apportionment for raw data...")
processed_data_raw_sa = calculate_source_apportionment(processed_data_raw)

# Calculate source apportionment for ONA-processed data
if 'Blue_BC_ONA' in processed_data_ona.columns and 'IR_BC_ONA' in processed_data_ona.columns:
    print("\nCalculating source apportionment for ONA-processed data...")
    processed_data_ona_sa = processed_data_ona.copy()
    # Use the ONA-processed data for source apportionment
    processed_data_ona_sa['Blue BC1'] = processed_data_ona['Blue_BC_ONA']
    processed_data_ona_sa['IR BC1'] = processed_data_ona['IR_BC_ONA']
    processed_data_ona_sa = calculate_source_apportionment(processed_data_ona_sa)
else:
    processed_data_ona_sa = processed_data_raw_sa.copy()
    print("\nWarning: ONA-processed data not available for both Blue and IR wavelengths, using raw data for source apportionment")

# Calculate source apportionment for CMA-processed data
print("\nCalculating source apportionment for CMA-processed data...")
processed_data_cma_sa = processed_data_cma.copy()
# Use the CMA-processed data for source apportionment
processed_data_cma_sa['Blue BC1'] = processed_data_cma['Blue_BC_CMA']
processed_data_cma_sa['IR BC1'] = processed_data_cma['IR_BC_CMA']
processed_data_cma_sa = calculate_source_apportionment(processed_data_cma_sa)

# Calculate source apportionment for DEMA-processed data
print("\nCalculating source apportionment for DEMA-processed data...")
processed_data_dema_sa = processed_data_dema.copy()
# Use the DEMA-processed data for source apportionment
processed_data_dema_sa['Blue BC1'] = processed_data_dema['Blue_BC_DEMA']
processed_data_dema_sa['IR BC1'] = processed_data_dema['IR_BC_DEMA'] 
processed_data_dema_sa = calculate_source_apportionment(processed_data_dema_sa)

## 9. Evaluate Processing Performance
  
Now let's evaluate how well each processing method performed and compare them.

In [ ]:
def evaluate_processing(data_raw, data_processed, raw_col, processed_col):
    """
    Evaluate the performance of a processing algorithm
    
    Parameters:
    -----------
    data_raw : pandas.DataFrame
        DataFrame with original data
    data_processed : pandas.DataFrame
        DataFrame with processed data
    raw_col : str
        Column name for raw data
    processed_col : str
        Column name for processed data
        
    Returns:
    --------
    metrics : dict
        Dictionary of performance metrics
    """
    # 1. Reduction of negatives
    numneg_raw = (data_raw[raw_col] < 0).sum() / len(data_raw)
    numneg_processed = (data_processed[processed_col] < 0).sum() / len(data_processed)
    
    print(f"Fraction of negative values in raw data: {numneg_raw:.4f}")
    print(f"Fraction of negative values after processing: {numneg_processed:.4f}")
    print(f"Reduction in negative values: {(numneg_raw - numneg_processed)/numneg_raw:.4f} ({(numneg_raw - numneg_processed)/numneg_raw*100:.1f}%)")
    
    # 2. Reduction of noise (average absolute difference between consecutive points)
    temp_raw = np.zeros(len(data_raw)-1)
    temp_processed = np.zeros(len(data_processed)-1)
    
    for i in range(len(data_raw)-1):
        temp_raw[i] = abs(data_raw[raw_col].iloc[i+1] - data_raw[raw_col].iloc[i])
        temp_processed[i] = abs(data_processed[processed_col].iloc[i+1] - data_processed[processed_col].iloc[i])
    
    noise_raw = np.nanmean(temp_raw)
    noise_processed = np.nanmean(temp_processed)
    
    print(f"Noise in raw data: {noise_raw:.1f} ng/m³")
    print(f"Noise in processed data: {noise_processed:.1f} ng/m³")
    print(f"Noise reduction factor: {noise_raw/noise_processed:.1f}x")
    
    # 3. Calculate correlation with raw data
    correlation = data_raw[raw_col].corr(data_processed[processed_col])
    print(f"Correlation with raw data: {correlation:.4f}")
    
    return {
        'negative_original': numneg_raw,
        'negative_processed': numneg_processed,
        'negative_reduction': (numneg_raw - numneg_processed)/numneg_raw if numneg_raw > 0 else 0,
        'noise_original': noise_raw,
        'noise_processed': noise_processed,
        'noise_reduction': noise_raw/noise_processed,
        'correlation': correlation
    }

In [ ]:
# Evaluate each wavelength and method
summary_metrics = {}

for wavelength in ['Blue', 'IR']:
    bc_col = f"{wavelength} BC1"
    ona_col = f"{wavelength}_BC_ONA"
    cma_col = f"{wavelength}_BC_CMA"
    dema_col = f"{wavelength}_BC_DEMA"
    
    if bc_col in data.columns:
        print(f"\n===== Performance Evaluation for {wavelength} Wavelength =====")
        
        # Evaluate ONA if available
        if ona_col in processed_data_ona.columns:
            print(f"\nEvaluating ONA for {wavelength} wavelength:")
            summary_metrics[f"{wavelength}_ONA"] = evaluate_processing(data, processed_data_ona, bc_col, ona_col)
        
        # Evaluate CMA
        print(f"\nEvaluating CMA for {wavelength} wavelength:")
        summary_metrics[f"{wavelength}_CMA"] = evaluate_processing(data, processed_data_cma, bc_col, cma_col)
        
        # Evaluate DEMA
        print(f"\nEvaluating DEMA for {wavelength} wavelength:")
        summary_metrics[f"{wavelength}_DEMA"] = evaluate_processing(data, processed_data_dema, bc_col, dema_col)

# Create a summary table
summary_table = pd.DataFrame.from_dict(summary_metrics, orient='index')
summary_table.columns = [
    'Negative values (original)',
    'Negative values (processed)', 
    'Negative reduction',
    'Noise (original ng/m³)', 
    'Noise (processed ng/m³)',
    'Noise reduction factor',
    'Correlation with raw'
]

# Display the summary table
print("\n===== Summary of Method Performance =====")
display(summary_table)

# Compare Source Apportionment Results
print("\n===== Source Apportionment Comparison =====")
print("\nComparison of BB% from Raw, ONA, CMA and DEMA processing:")
print(f"Raw data - Mean BB%: {processed_data_raw_sa['BB_Percent'].mean():.2f}, Std: {processed_data_raw_sa['BB_Percent'].std():.2f}")

if 'BB_Percent' in processed_data_ona_sa.columns:
    print(f"ONA data - Mean BB%: {processed_data_ona_sa['BB_Percent'].mean():.2f}, Std: {processed_data_ona_sa['BB_Percent'].std():.2f}")

print(f"CMA data - Mean BB%: {processed_data_cma_sa['BB_Percent'].mean():.2f}, Std: {processed_data_cma_sa['BB_Percent'].std():.2f}")
print(f"DEMA data - Mean BB%: {processed_data_dema_sa['BB_Percent'].mean():.2f}, Std: {processed_data_dema_sa['BB_Percent'].std():.2f}")

## 10. Visualize Results
  
Let's visualize the raw and processed data to see the effects of our algorithms.

In [ ]:
def plot_comparison(data, data_ona, data_cma, data_dema, wavelength, sample_period=None):
    """
    Plot the raw, ONA, CMA, and DEMA processed BC data for comparison
    
    Parameters:
    -----------
    data : pandas.DataFrame
        DataFrame with raw data
    data_ona : pandas.DataFrame
        DataFrame with ONA processed data
    data_cma : pandas.DataFrame
        DataFrame with CMA processed data
    data_dema : pandas.DataFrame
        DataFrame with DEMA processed data
    wavelength : str
        Which wavelength to plot
    sample_period : tuple, optional
        Start and end indices for a subset of the data to plot
    """
    # Identify columns
    bc_col = f"{wavelength} BC1"
    ona_col = f"{wavelength}_BC_ONA"
    cma_col = f"{wavelength}_BC_CMA"
    dema_col = f"{wavelength}_BC_DEMA"
    
    # Check which methods are available
    methods_available = []
    if bc_col in data.columns:
        methods_available.append(('Raw', data, bc_col, 'k-'))
    if ona_col in data_ona.columns:
        methods_available.append(('ONA', data_ona, ona_col, 'b-'))
    if cma_col in data_cma.columns:
        methods_available.append(('CMA', data_cma, cma_col, 'r-'))
    if dema_col in data_dema.columns:
        methods_available.append(('DEMA', data_dema, dema_col, 'g-'))
    
    # Select a subset of data if specified
    if sample_period is not None:
        start_idx, end_idx = sample_period
        plot_datas = []
        for name, df, col, style in methods_available:
            plot_datas.append((name, df.iloc[start_idx:end_idx].copy(), col, style))
    else:
        plot_datas = methods_available
    
    # Create a figure
    plt.figure(figsize=(12, 6))
    
    # Create x-axis values
    if len(plot_datas) > 0 and 'Time (UTC)' in plot_datas[0][1].columns:
        try:
            x = pd.to_datetime(plot_datas[0][1]['Time (UTC)'])
            x_formatter = mdates.DateFormatter('%H:%M')
            plt.gca().xaxis.set_major_formatter(x_formatter)
            plt.gcf().autofmt_xdate()
            x_label = 'Time (UTC)'
        except:
            x = np.arange(len(plot_datas[0][1]))
            x_label = 'Data Point'
    else:
        x = np.arange(len(plot_datas[0][1]) if len(plot_datas) > 0 else 0)
        x_label = 'Data Point'
    
    # Plot BC data for each method
    for name, df, col, style in plot_datas:
        if name == 'Raw':
            plt.plot(x, df[col], style, alpha=0.5, label=name)
        else:
            plt.plot(x, df[col], style, label=name)
    
    plt.xlabel(x_label)
    plt.ylabel(f'{wavelength} BC (ng/m³)')
    plt.title(f'Comparison of Noise Reduction Methods for {wavelength} Wavelength')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_source_apportionment_comparison(data_raw, data_ona, data_cma, data_dema, sample_period=None):
    """
    Plot the source apportionment results for raw, ONA, CMA and DEMA data
    
    Parameters:
    -----------
    data_raw : pandas.DataFrame
        DataFrame with raw source apportionment data
    data_ona : pandas.DataFrame
        DataFrame with ONA processed source apportionment data
    data_cma : pandas.DataFrame
        DataFrame with CMA processed source apportionment data
    data_dema : pandas.DataFrame
        DataFrame with DEMA processed source apportionment data
    sample_period : tuple, optional
        Start and end indices for a subset of the data to plot
    """
    # Check which methods have source apportionment data
    methods_available = []
    if 'BB_Percent' in data_raw.columns:
        methods_available.append(('Raw', data_raw, 'BB_Percent', 'k-'))
    if 'BB_Percent' in data_ona.columns:
        methods_available.append(('ONA', data_ona, 'BB_Percent', 'b-'))
    if 'BB_Percent' in data_cma.columns:
        methods_available.append(('CMA', data_cma, 'BB_Percent', 'r-'))
    if 'BB_Percent' in data_dema.columns:
        methods_available.append(('DEMA', data_dema, 'BB_Percent', 'g-'))
    
    # Select a subset of data if specified
    if sample_period is not None:
        start_idx, end_idx = sample_period
        plot_datas = []
        for name, df, col, style in methods_available:
            plot_datas.append((name, df.iloc[start_idx:end_idx].copy(), col, style))
    else:
        plot_datas = methods_available
    
    # Create a figure with three subplots
    fig, axes = plt.subplots(3, 1, figsize=(12, 12), sharex=True)
    
    # Create x-axis values
    if len(plot_datas) > 0 and 'Time (UTC)' in plot_datas[0][1].columns:
        try:
            x = pd.to_datetime(plot_datas[0][1]['Time (UTC)'])
            x_formatter = mdates.DateFormatter('%H:%M')
            x_label = 'Time (UTC)'
            for ax in axes:
                ax.xaxis.set_major_formatter(x_formatter)
            fig.autofmt_xdate()
        except:
            x = np.arange(len(plot_datas[0][1]))
            x_label = 'Data Point'
    else:
        x = np.arange(len(plot_datas[0][1]) if len(plot_datas) > 0 else 0)
        x_label = 'Data Point'
    
    # Plot BB percentage for each method
    for name, df, col, style in plot_datas:
        if name == 'Raw':
            axes[0].plot(x, df[col], style, alpha=0.5, label=name)
        else:
            axes[0].plot(x, df[col], style, label=name)
    
    axes[0].set_ylabel('Biomass Burning %')
    axes[0].set_title('Source Apportionment Results Comparison')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Plot Wood Burning BC for each method
    for name, df, style in [(name, df, style) for name, df, col, style in plot_datas]:
        if 'BC_WB' in df.columns:
            if name == 'Raw':
                axes[1].plot(x, df['BC_WB'], style, alpha=0.5, label=name)
            else:
                axes[1].plot(x, df['BC_WB'], style, label=name)
    
    axes[1].set_ylabel('Wood Burning BC (ng/m³)')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    # Plot Fossil Fuel BC for each method
    for name, df, style in [(name, df, style) for name, df, col, style in plot_datas]:
        if 'BC_FF' in df.columns:
            if name == 'Raw':
                axes[2].plot(x, df['BC_FF'], style, alpha=0.5, label=name)
            else:
                axes[2].plot(x, df['BC_FF'], style, label=name)
    
    axes[2].set_ylabel('Fossil Fuel BC (ng/m³)')
    axes[2].set_xlabel(x_label)
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Let's plot the comparison of all methods for each wavelength

In [ ]:
# Plot comparison for each wavelength
for wavelength in ['Blue', 'IR']:
    bc_col = f"{wavelength} BC1"
    ona_col = f"{wavelength}_BC_ONA"
    cma_col = f"{wavelength}_BC_CMA"
    dema_col = f"{wavelength}_BC_DEMA"
    
    if bc_col in data.columns:
        print(f"\nPlots for {wavelength} wavelength comparison:")
        
        # Plot full dataset
        plot_comparison(data, processed_data_ona, processed_data_cma, processed_data_dema, wavelength)
        
        # Plot a sample period (first 1000 points or 10% of data, whichever is smaller)
        sample_size = min(1000, int(len(data) * 0.1))
        if sample_size < len(data):
            print(f"\nZoomed view of first {sample_size} points:")
            plot_comparison(data, processed_data_ona, processed_data_cma, processed_data_dema, wavelength, (0, sample_size))

# Plot source apportionment comparisons
print("\nSource apportionment comparison plots:")
plot_source_apportionment_comparison(processed_data_raw_sa, processed_data_ona_sa, processed_data_cma_sa, processed_data_dema_sa)

# Plot a sample period
sample_size = min(1000, int(len(data) * 0.1))
if sample_size < len(data):
    print(f"\nZoomed view of first {sample_size} points:")
    plot_source_apportionment_comparison(processed_data_raw_sa, processed_data_ona_sa, processed_data_cma_sa, processed_data_dema_sa, (0, sample_size))


## 11. Side-by-Side Visualization Comparison
  
Now let's create a side-by-side comparison of all methods for easier visual comparison.

In [ ]:
def plot_side_by_side_comparison(data, data_ona, data_cma, data_dema, wavelength='Blue', sample_period=None, timebase=60):
    """
    Plot raw, ONA, CMA, and DEMA processed BC data in a side-by-side grid layout
    
    Parameters:
    -----------
    data : pandas.DataFrame
        DataFrame with raw data
    data_ona : pandas.DataFrame
        DataFrame with ONA processed data
    data_cma : pandas.DataFrame
        DataFrame with CMA processed data
    data_dema : pandas.DataFrame
        DataFrame with DEMA processed data
    wavelength : str
        Which wavelength to plot
    sample_period : tuple, optional
        Start and end indices for a subset of the data to plot
    timebase : int
        The timebase in seconds (5, 10, 30, 60, etc.)
    """
    # Identify columns
    bc_col = f"{wavelength} BC1"
    ona_col = f"{wavelength}_BC_ONA"
    cma_col = f"{wavelength}_BC_CMA"
    dema_col = f"{wavelength}_BC_DEMA"
    
    # Determine which methods are available
    methods = []
    if bc_col in data.columns:
        methods.append(('Raw', data, bc_col))
    if ona_col in data_ona.columns:
        methods.append(('ONA', data_ona, ona_col))
    if cma_col in data_cma.columns:
        methods.append(('CMA', data_cma, cma_col))
    if dema_col in data_dema.columns:
        methods.append(('DEMA', data_dema, dema_col))
    
    # Select a subset of data if specified
    if sample_period is not None:
        start_idx, end_idx = sample_period
        plot_methods = []
        for name, df, col in methods:
            plot_methods.append((name, df.iloc[start_idx:end_idx].copy(), col))
    else:
        plot_methods = methods
    
    # Determine number of panels needed (number of methods + 1 for comparison)
    n_methods = len(plot_methods)
    if n_methods < 2:
        print("Not enough methods to compare")
        return
    
    # Calculate grid dimensions (approximately square)
    n_rows = int(np.ceil(np.sqrt(n_methods + 1)))
    n_cols = int(np.ceil((n_methods + 1) / n_rows))
    
    # Create a figure with a grid of subplots
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols*5, n_rows*4), sharex=True, sharey=True)
    axes = axes.flatten()  # Flatten for easier indexing
    
    # Create x-axis values
    if 'Time (UTC)' in plot_methods[0][1].columns:
        try:
            x = pd.to_datetime(plot_methods[0][1]['Time (UTC)'])
            x_formatter = mdates.DateFormatter('%H:%M')
            x_label = 'Time (UTC)'
            for ax in axes:
                ax.xaxis.set_major_formatter(x_formatter)
            fig.autofmt_xdate()
        except:
            x = np.arange(len(plot_methods[0][1]))
            x_label = 'Data Point'
    else:
        x = np.arange(len(plot_methods[0][1]))
        x_label = 'Data Point'
    
    # Set y-limits for consistent comparison
    all_y_values = []
    for name, plot_data, col in plot_methods:
        values = plot_data[col].dropna()
        if len(values) > 0:
            all_y_values.extend(values)
    
    if all_y_values:
        ymin = np.percentile(all_y_values, 1)  # 1st percentile to avoid extreme outliers
        ymax = np.percentile(all_y_values, 99)  # 99th percentile to avoid extreme outliers
        # Add a small buffer to the y-limits for visual clarity
        y_buffer = (ymax - ymin) * 0.1
        ylim = (ymin - y_buffer, ymax + y_buffer)
    else:
        ylim = (0, 1)  # Default if no valid values
    
    # Plot titles and data for individual methods
    for i, (name, plot_data, col) in enumerate(plot_methods):
        title = f'{name} processed ({timebase}s)'
        if name == 'Raw':
            title = f'Raw data ({timebase}s)'
            axes[i].plot(x, plot_data[col], 'k-', label=name)
        elif name == 'ONA':
            axes[i].plot(x, plot_data[col], 'b-', label=name)
        elif name == 'CMA':
            axes[i].plot(x, plot_data[col], 'r-', label=name)
        elif name == 'DEMA':
            axes[i].plot(x, plot_data[col], 'g-', label=name)
        
        axes[i].set_title(title)
        axes[i].set_ylabel(f'{wavelength} BC (ng/m³)')
        axes[i].set_ylim(ylim)
        axes[i].grid(True, alpha=0.3)
    
    # Plot all methods for comparison in the last panel
    for name, plot_data, col in plot_methods:
        if name == 'Raw':
            axes[-1].plot(x, plot_data[col], 'k-', alpha=0.5, label=name)
        elif name == 'ONA':
            axes[-1].plot(x, plot_data[col], 'b-', label=name)
        elif name == 'CMA':
            axes[-1].plot(x, plot_data[col], 'r-', label=name)
        elif name == 'DEMA':
            axes[-1].plot(x, plot_data[col], 'g-', label=name)
    
    axes[-1].set_title('All methods comparison')
    axes[-1].set_ylabel(f'{wavelength} BC (ng/m³)')
    axes[-1].set_xlabel(x_label)
    axes[-1].set_ylim(ylim)
    axes[-1].grid(True, alpha=0.3)
    axes[-1].legend()
    
    # Hide any extra subplots
    for i in range(n_methods + 1, len(axes)):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    plt.suptitle(f'Side-by-Side Comparison of Processing Methods ({wavelength} Wavelength)', y=1.02, fontsize=16)
    plt.show()

In [ ]:
def plot_source_apportionment_side_by_side(data_raw, data_ona, data_cma, data_dema, sample_period=None, timebase=60):
    """
    Plot source apportionment results in a side-by-side grid layout
    
    Parameters:
    -----------
    data_raw : pandas.DataFrame
        DataFrame with raw source apportionment data
    data_ona : pandas.DataFrame
        DataFrame with ONA processed source apportionment data
    data_cma : pandas.DataFrame
        DataFrame with CMA processed source apportionment data
    data_dema : pandas.DataFrame
        DataFrame with DEMA processed source apportionment data
    sample_period : tuple, optional
        Start and end indices for a subset of the data to plot
    timebase : int
        The timebase in seconds (5, 10, 30, 60, etc.)
    """
    # Determine which methods have source apportionment data
    methods = []
    if 'BB_Percent' in data_raw.columns:
        methods.append(('Raw', data_raw))
    if 'BB_Percent' in data_ona.columns:
        methods.append(('ONA', data_ona))
    if 'BB_Percent' in data_cma.columns:
        methods.append(('CMA', data_cma))
    if 'BB_Percent' in data_dema.columns:
        methods.append(('DEMA', data_dema))
    
    # Select a subset of data if specified
    if sample_period is not None:
        start_idx, end_idx = sample_period
        plot_methods = []
        for name, df in methods:
            plot_methods.append((name, df.iloc[start_idx:end_idx].copy()))
    else:
        plot_methods = methods
    
    # Determine number of panels needed (number of methods + 1 for comparison)
    n_methods = len(plot_methods)
    if n_methods < 2:
        print("Not enough methods to compare")
        return
    
    # Calculate grid dimensions (approximately square)
    n_rows = int(np.ceil(np.sqrt(n_methods + 1)))
    n_cols = int(np.ceil((n_methods + 1) / n_rows))
    
    # Create a figure with a grid of subplots
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols*5, n_rows*4), sharex=True, sharey=True)
    axes = axes.flatten()  # Flatten for easier indexing
    
    # Create x-axis values
    if 'Time (UTC)' in plot_methods[0][1].columns:
        try:
            x = pd.to_datetime(plot_methods[0][1]['Time (UTC)'])
            x_formatter = mdates.DateFormatter('%H:%M')
            x_label = 'Time (UTC)'
            for ax in axes:
                ax.xaxis.set_major_formatter(x_formatter)
            fig.autofmt_xdate()
        except:
            x = np.arange(len(plot_methods[0][1]))
            x_label = 'Data Point'
    else:
        x = np.arange(len(plot_methods[0][1]))
        x_label = 'Data Point'
    
    # Set y-limits for BB percentage for consistent comparison
    all_bb_values = []
    for name, plot_data in plot_methods:
        values = plot_data['BB_Percent'].dropna()
        if len(values) > 0:
            all_bb_values.extend(values)
    
    if all_bb_values:
        ymin_bb = max(0, np.percentile(all_bb_values, 1))  # 1st percentile to avoid extreme outliers, but not below 0
        ymax_bb = min(100, np.percentile(all_bb_values, 99))  # 99th percentile to avoid extreme outliers, but not above 100
        # Add a small buffer to the y-limits for visual clarity
        y_buffer_bb = (ymax_bb - ymin_bb) * 0.1
        ylim_bb = (max(0, ymin_bb - y_buffer_bb), min(100, ymax_bb + y_buffer_bb))
    else:
        ylim_bb = (0, 100)  # Default range for BB%
    
    # Plot titles and data for individual methods
    for i, (name, plot_data) in enumerate(plot_methods):
        title = f'{name} processed ({timebase}s)'
        if name == 'Raw':
            title = f'Raw data ({timebase}s)'
            axes[i].plot(x, plot_data['BB_Percent'], 'k-', label=name)
        elif name == 'ONA':
            axes[i].plot(x, plot_data['BB_Percent'], 'b-', label=name)
        elif name == 'CMA':
            axes[i].plot(x, plot_data['BB_Percent'], 'r-', label=name)
        elif name == 'DEMA':
            axes[i].plot(x, plot_data['BB_Percent'], 'g-', label=name)
        
        axes[i].set_title(title)
        axes[i].set_ylabel('Biomass Burning %')
        axes[i].set_ylim(ylim_bb)
        axes[i].grid(True, alpha=0.3)
    
    # Plot all methods for comparison in the last panel
    for name, plot_data in plot_methods:
        if name == 'Raw':
            axes[-1].plot(x, plot_data['BB_Percent'], 'k-', alpha=0.5, label=name)
        elif name == 'ONA':
            axes[-1].plot(x, plot_data['BB_Percent'], 'b-', label=name)
        elif name == 'CMA':
            axes[-1].plot(x, plot_data['BB_Percent'], 'r-', label=name)
        elif name == 'DEMA':
            axes[-1].plot(x, plot_data['BB_Percent'], 'g-', label=name)
    
    axes[-1].set_title('All methods comparison')
    axes[-1].set_ylabel('Biomass Burning %')
    axes[-1].set_xlabel(x_label)
    axes[-1].set_ylim(ylim_bb)
    axes[-1].grid(True, alpha=0.3)
    axes[-1].legend()
    
    # Hide any extra subplots
    for i in range(n_methods + 1, len(axes)):
        axes[i].set_visible(False)
    
    if x_label == 'Time (UTC)':
        fig.autofmt_xdate()
    
    plt.tight_layout()
    plt.suptitle('Side-by-Side Comparison of Source Apportionment Results', y=1.02, fontsize=16)
    plt.show()

In [ ]:
# Determine the timebase from the data
if 'Timebase (s)' in data.columns:
    timebase = data['Timebase (s)'].iloc[0]
else:
    timebase = 60  # default assumption

# Plot side-by-side comparison for each wavelength
for wavelength in ['Blue', 'IR']:
    bc_col = f"{wavelength} BC1"
    
    if bc_col in data.columns:
        print(f"\nSide-by-side comparison for {wavelength} wavelength:")
        
        # Plot full dataset
        plot_side_by_side_comparison(data, processed_data_ona, processed_data_cma, processed_data_dema, 
                                  wavelength, timebase=timebase)
        
        # Plot a sample period (first 1000 points or 10% of data, whichever is smaller)
        sample_size = min(1000, int(len(data) * 0.1))
        if sample_size < len(data):
            print(f"\nZoomed view of first {sample_size} points:")
            plot_side_by_side_comparison(data, processed_data_ona, processed_data_cma, processed_data_dema, 
                                      wavelength, (0, sample_size), timebase=timebase)

# Plot source apportionment comparisons
print("\nSide-by-side source apportionment comparison:")
plot_source_apportionment_side_by_side(processed_data_raw_sa, processed_data_ona_sa, processed_data_cma_sa, 
                                     processed_data_dema_sa, timebase=timebase)

# Plot a sample period for source apportionment
sample_size = min(1000, int(len(data) * 0.1))
if sample_size < len(data):
    print(f"\nZoomed view of first {sample_size} points for source apportionment:")
    plot_source_apportionment_side_by_side(processed_data_raw_sa, processed_data_ona_sa, processed_data_cma_sa, 
                                        processed_data_dema_sa, (0, sample_size), timebase=timebase)


## 12. Stacked Temporal Comparison
  
Let's also create a visualization that stacks the temporal plots vertically for each processing method.

In [ ]:
def plot_stacked_temporal_comparison(data, data_ona, data_cma, data_dema, wavelength='Blue', sample_period=None):
    """
    Plot stacked temporal comparison of raw, ONA, CMA, and DEMA processed BC data
    
    Parameters:
    -----------
    data : pandas.DataFrame
        DataFrame with raw data
    data_ona : pandas.DataFrame
        DataFrame with ONA processed data
    data_cma : pandas.DataFrame
        DataFrame with CMA processed data
    data_dema : pandas.DataFrame
        DataFrame with DEMA processed data
    wavelength : str
        Which wavelength to plot
    sample_period : tuple, optional
        Start and end indices for a subset of the data to plot
    """
    # Identify columns
    bc_col = f"{wavelength} BC1"
    ona_col = f"{wavelength}_BC_ONA"
    cma_col = f"{wavelength}_BC_CMA"
    dema_col = f"{wavelength}_BC_DEMA"
    
    # Determine which methods are available
    methods = []
    if bc_col in data.columns:
        methods.append(('Raw', data, bc_col))
    if ona_col in data_ona.columns:
        methods.append(('ONA', data_ona, ona_col))
    if cma_col in data_cma.columns:
        methods.append(('CMA', data_cma, cma_col))
    if dema_col in data_dema.columns:
        methods.append(('DEMA', data_dema, dema_col))
    
    # Select a subset of data if specified
    if sample_period is not None:
        start_idx, end_idx = sample_period
        plot_methods = []
        for name, df, col in methods:
            plot_methods.append((name, df.iloc[start_idx:end_idx].copy(), col))
    else:
        plot_methods = methods
    
    # Determine number of panels needed
    n_methods = len(plot_methods)
    if n_methods < 2:
        print("Not enough methods to compare")
        return
    
    # Create a figure with vertically stacked subplots
    fig, axes = plt.subplots(n_methods, 1, figsize=(12, n_methods*3), sharex=True)
    if n_methods == 1:
        axes = [axes]  # Make it a list for consistent indexing
    
    # Determine the timebase
    if 'Timebase (s)' in plot_methods[0][1].columns:
        timebase = plot_methods[0][1]['Timebase (s)'].iloc[0]
    else:
        timebase = 60  # default
    
    # Create x-axis values
    if 'Time (UTC)' in plot_methods[0][1].columns:
        try:
            x = pd.to_datetime(plot_methods[0][1]['Time (UTC)'])
            x_formatter = mdates.DateFormatter('%H:%M')
            x_label = 'Time (UTC)'
            for ax in axes:
                ax.xaxis.set_major_formatter(x_formatter)
            fig.autofmt_xdate()
        except:
            x = np.arange(len(plot_methods[0][1]))
            x_label = 'Data Point'
    else:
        x = np.arange(len(plot_methods[0][1]))
        x_label = 'Data Point'
    
    # Set y-limits for consistent comparison
    all_y_values = []
    for name, plot_data, col in plot_methods:
        values = plot_data[col].dropna()
        if len(values) > 0:
            all_y_values.extend(values)
    
    if all_y_values:
        ymin = np.percentile(all_y_values, 1)  # 1st percentile to avoid extreme outliers
        ymax = np.percentile(all_y_values, 99)  # 99th percentile to avoid extreme outliers
        # Ensure min and max are different to prevent division by zero
        if ymin == ymax:
            ymin -= 10
            ymax += 10
            
        y_buffer = (ymax - ymin) * 0.1
        ylim = (ymin - y_buffer, ymax + y_buffer)
    else:
        ylim = (0, 1)  # Default if no valid values
    
    # Plot data for each method
    for i, (name, plot_data, col) in enumerate(plot_methods):
        if name == 'Raw':
            axes[i].plot(x, plot_data[col], 'k-', label=f'Raw ({timebase}s)')
        elif name == 'ONA':
            axes[i].plot(x, plot_data[col], 'b-', label=f'ONA ({timebase}s)')
        elif name == 'CMA':
            axes[i].plot(x, plot_data[col], 'r-', label=f'CMA ({timebase}s)')
        elif name == 'DEMA':
            axes[i].plot(x, plot_data[col], 'g-', label=f'DEMA ({timebase}s)')
        
        axes[i].set_ylabel(f'{wavelength} BC (ng/m³)')
        axes[i].legend(loc='upper right')
        axes[i].grid(True, alpha=0.3)
        axes[i].set_ylim(ylim)
    
    axes[-1].set_xlabel(x_label)
    plt.suptitle(f'Stacked Temporal Comparison ({wavelength} Wavelength)', fontsize=16)
    plt.tight_layout()
    plt.subplots_adjust(top=0.95)  # Adjust for suptitle
    plt.show()

In [ ]:
def plot_stacked_source_apportionment(data_raw, data_ona, data_cma, data_dema, sample_period=None):
    """
    Plot stacked temporal comparison of source apportionment results
    
    Parameters:
    -----------
    data_raw : pandas.DataFrame
        DataFrame with raw source apportionment data
    data_ona : pandas.DataFrame
        DataFrame with ONA processed source apportionment data
    data_cma : pandas.DataFrame
        DataFrame with CMA processed source apportionment data
    data_dema : pandas.DataFrame
        DataFrame with DEMA processed source apportionment data
    sample_period : tuple, optional
        Start and end indices for a subset of the data to plot
    """
    # Determine which methods have source apportionment data
    methods = []
    if 'BB_Percent' in data_raw.columns:
        methods.append(('Raw', data_raw))
    if 'BB_Percent' in data_ona.columns:
        methods.append(('ONA', data_ona))
    if 'BB_Percent' in data_cma.columns:
        methods.append(('CMA', data_cma))
    if 'BB_Percent' in data_dema.columns:
        methods.append(('DEMA', data_dema))
    
    # Select a subset of data if specified
    if sample_period is not None:
        start_idx, end_idx = sample_period
        plot_methods = []
        for name, df in methods:
            plot_methods.append((name, df.iloc[start_idx:end_idx].copy()))
    else:
        plot_methods = methods
    
    # Determine number of panels needed
    n_methods = len(plot_methods)
    if n_methods < 2:
        print("Not enough methods to compare")
        return
    
    # Create a figure with vertically stacked subplots
    fig, axes = plt.subplots(n_methods, 1, figsize=(12, n_methods*3), sharex=True)
    if n_methods == 1:
        axes = [axes]  # Make it a list for consistent indexing
    
    # Determine the timebase
    if 'Timebase (s)' in plot_methods[0][1].columns:
        timebase = plot_methods[0][1]['Timebase (s)'].iloc[0]
    else:
        timebase = 60  # default
    
    # Create x-axis values
    if 'Time (UTC)' in plot_methods[0][1].columns:
        try:
            x = pd.to_datetime(plot_methods[0][1]['Time (UTC)'])
            x_formatter = mdates.DateFormatter('%H:%M')
            x_label = 'Time (UTC)'
            for ax in axes:
                ax.xaxis.set_major_formatter(x_formatter)
            fig.autofmt_xdate()
        except:
            x = np.arange(len(plot_methods[0][1]))
            x_label = 'Data Point'
    else:
        x = np.arange(len(plot_methods[0][1]))
        x_label = 'Data Point'
    
    # Set y-limits for BB percentage for consistent comparison
    all_y_values = []
    for name, plot_data in plot_methods:
        values = plot_data['BB_Percent'].dropna()
        if len(values) > 0:
            all_y_values.extend(values)
    
    if all_y_values:
        ymin = max(0, np.percentile(all_y_values, 1))  # 1st percentile but not below 0
        ymax = min(100, np.percentile(all_y_values, 99))  # 99th percentile but not above 100
        
        # Ensure min and max are different to prevent division by zero
        if ymin == ymax:
            if ymin == 0:
                ymax = 10  # If all zeros, set max to 10%
            elif ymax == 100:
                ymin = 90  # If all 100s, set min to 90%
            else:
                # Otherwise add/subtract 5 percentage points
                ymin = max(0, ymin - 5)
                ymax = min(100, ymax + 5)
            
        y_buffer = (ymax - ymin) * 0.1
        ylim = (max(0, ymin - y_buffer), min(100, ymax + y_buffer))
    else:
        ylim = (0, 100)  # Default range for BB%
    
    # Plot data for each method
    for i, (name, plot_data) in enumerate(plot_methods):
        if name == 'Raw':
            axes[i].plot(x, plot_data['BB_Percent'], 'k-', label=f'Raw ({timebase}s)')
        elif name == 'ONA':
            axes[i].plot(x, plot_data['BB_Percent'], 'b-', label=f'ONA ({timebase}s)')
        elif name == 'CMA':
            axes[i].plot(x, plot_data['BB_Percent'], 'r-', label=f'CMA ({timebase}s)')
        elif name == 'DEMA':
            axes[i].plot(x, plot_data['BB_Percent'], 'g-', label=f'DEMA ({timebase}s)')
        
        axes[i].set_ylabel('Biomass Burning %')
        axes[i].legend(loc='upper right')
        axes[i].grid(True, alpha=0.3)
        axes[i].set_ylim(ylim)
    
    axes[-1].set_xlabel(x_label)
    plt.suptitle('Stacked Temporal Comparison of Source Apportionment Results', fontsize=16)
    plt.tight_layout()
    plt.subplots_adjust(top=0.95)  # Adjust for suptitle
    plt.show()

In [ ]:
# Plot stacked temporal comparison for each wavelength
for wavelength in ['Blue', 'IR']:
    bc_col = f"{wavelength} BC1"
    ona_col = f"{wavelength}_BC_ONA"
    cma_col = f"{wavelength}_BC_CMA"
    dema_col = f"{wavelength}_BC_DEMA"
    
    # Check if required columns exist
    methods_available = []
    if bc_col in data.columns:
        methods_available.append('Raw')
    if ona_col in processed_data_ona.columns:
        methods_available.append('ONA')
    if cma_col in processed_data_cma.columns:
        methods_available.append('CMA')
    if dema_col in processed_data_dema.columns:
        methods_available.append('DEMA')
    
    # Only proceed if at least two methods are available
    if len(methods_available) >= 2:
        print(f"\nStacked temporal comparison for {wavelength} wavelength:")
        
        # Plot full dataset
        plot_stacked_temporal_comparison(data, processed_data_ona, processed_data_cma, processed_data_dema, wavelength)
        
        # Plot a sample period (first 1000 points or 10% of data, whichever is smaller)
        sample_size = min(1000, int(len(data) * 0.1))
        if sample_size < len(data):
            print(f"\nZoomed view of first {sample_size} points:")
            plot_stacked_temporal_comparison(data, processed_data_ona, processed_data_cma, processed_data_dema, 
                                          wavelength, (0, sample_size))

# Plot stacked source apportionment comparisons
# Check if source apportionment data is available for at least two methods
sa_methods_available = []
if 'BB_Percent' in processed_data_raw_sa.columns:
    sa_methods_available.append('Raw')
if 'BB_Percent' in processed_data_ona_sa.columns:
    sa_methods_available.append('ONA')
if 'BB_Percent' in processed_data_cma_sa.columns:
    sa_methods_available.append('CMA')
if 'BB_Percent' in processed_data_dema_sa.columns:
    sa_methods_available.append('DEMA')

if len(sa_methods_available) >= 2:
    print("\nStacked source apportionment comparison:")
    plot_stacked_source_apportionment(processed_data_raw_sa, processed_data_ona_sa, processed_data_cma_sa, processed_data_dema_sa)

    # Plot a sample period for source apportionment
    sample_size = min(1000, int(len(data) * 0.1))
    if sample_size < len(data):
        print(f"\nZoomed view of first {sample_size} points for source apportionment:")
        plot_stacked_source_apportionment(processed_data_raw_sa, processed_data_ona_sa, processed_data_cma_sa, 
                                       processed_data_dema_sa, (0, sample_size))


## 13. Time Averaging Analysis
 
Let's analyze the time averaging behavior of the ONA method and compare it to the effective averaging window of the CMA and DEMA methods.

In [ ]:
def analyze_time_averaging(data_ona, data_cma, data_dema, wavelength='Blue'):
    """
    Analyze and compare the time averaging behavior of each method
    
    Parameters:
    -----------
    data_ona : pandas.DataFrame
        DataFrame with ONA processed data
    data_cma : pandas.DataFrame
        DataFrame with CMA processed data
    data_dema : pandas.DataFrame
        DataFrame with DEMA processed data
    wavelength : str
        Which wavelength to analyze
    """
    # Check which methods are available
    methods = []
    
    # For ONA, check if points_averaged column exists
    points_averaged_col = f"{wavelength}_points_averaged"
    if points_averaged_col in data_ona.columns:
        # ONA explicit averaging window
        methods.append(('ONA', data_ona[points_averaged_col]))
    
    # For CMA, we know the window size
    if f"{wavelength}_BC_CMA" in data_cma.columns:
        # Get the fixed window size for CMA
        if 'Timebase (s)' in data_cma.columns:
            timebase = data_cma['Timebase (s)'].iloc[0]
            if timebase == 1:
                cma_window = 11  # 11 seconds for 1-second data
            elif timebase == 5:
                cma_window = 5   # 25 seconds for 5-second data
            elif timebase == 60:
                cma_window = 3   # 3 minutes for 1-minute data
            else:
                cma_window = 5   # Default for other timebases
        else:
            cma_window = 5       # Default if timebase is unknown
        
        # Make sure window_size is odd
        if cma_window % 2 == 0:
            cma_window += 1
            
        # Create a Series with the fixed window size
        cma_points = pd.Series([cma_window] * len(data_cma))
        methods.append(('CMA', cma_points))
    
    # For DEMA, calculate the equivalent window size based on alpha
    if f"{wavelength}_BC_DEMA" in data_dema.columns:
        # Calculate equivalent window size from alpha
        # For DEMA with alpha, N = 2/alpha - 1
        if 'Timebase (s)' in data_dema.columns:
            timebase = data_dema['Timebase (s)'].iloc[0]
            
            # Default alphas based on timebase
            if timebase == 1:
                alpha = 2 / (300 / timebase + 1)  # ~5-minute window for 1-second data
            elif timebase == 5:
                alpha = 2 / (300 / timebase + 1)  # ~5-minute window for 5-second data
            elif timebase == 60:
                alpha = 2 / (900 / timebase + 1)  # ~15-minute window for 60-second data
            else:
                alpha = 0.125  # Default
            
            # Equivalent window size for DEMA
            dema_equiv_window = int(2/alpha - 1)
            dema_points = pd.Series([dema_equiv_window] * len(data_dema))
            methods.append(('DEMA', dema_points))
    
    if len(methods) == 0:
        print("No time averaging data available for analysis")
        return
    
    # Create a figure
    plt.figure(figsize=(12, 6))
    
    # Plot histograms for each method on the same plot
    for name, points in methods:
        if name == 'ONA':
            plt.hist(points, bins=30, alpha=0.7, label=f'ONA (adaptive, mean={points.mean():.1f})')
        elif name == 'CMA':
            plt.hist(points, bins=30, alpha=0.7, label=f'CMA (fixed, window={cma_window})')
        elif name == 'DEMA':
            plt.hist(points, bins=30, alpha=0.7, label=f'DEMA (equiv. window={dema_equiv_window})')
    
    plt.xlabel('Number of Points Averaged')
    plt.ylabel('Frequency')
    plt.title(f'Distribution of Averaging Window Size - {wavelength} Wavelength')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Create a table with summary statistics
    summary_stats = {}
    for name, points in methods:
        summary_stats[name] = {
            'Mean': points.mean(),
            'Median': points.median(),
            'Min': points.min(),
            'Max': points.max(),
            'Std Dev': points.std()
        }
    
    # Convert to DataFrame for display
    summary_df = pd.DataFrame.from_dict(summary_stats, orient='index')
    print(f"\nSummary statistics of window sizes for {wavelength} wavelength:")
    display(summary_df)
    
    # If ONA is available, plot ECDF
    ona_points = None
    for name, points in methods:
        if name == 'ONA':
            ona_points = points
            break
    
    if ona_points is not None:
        plt.figure(figsize=(10, 6))
        
        # Calculate ECDF
        x = np.sort(ona_points)
        y = np.arange(1, len(x) + 1) / len(x)
        
        # Plot ECDF
        plt.semilogx(x, y)
        plt.xlabel('Number of Points Averaged')
        plt.ylabel('Fraction of Time Series ≤ Points')
        plt.title(f'ECDF of ONA Adaptive Window Size - {wavelength} Wavelength')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()


In [ ]:
# Analyze time averaging for each wavelength
for wavelength in ['Blue', 'IR']:
    print(f"\nTime averaging analysis for {wavelength} wavelength:")
    analyze_time_averaging(processed_data_ona, processed_data_cma, processed_data_dema, wavelength)


## 14. Comparing Signal Preservation

Let's examine how well each method preserves important signal features by looking at cross-correlation and lag.

In [ ]:
def analyze_signal_preservation(data_raw, data_ona, data_cma, data_dema, wavelength='Blue'):
    """
    Analyze how well each method preserves important signal features
    
    Parameters:
    -----------
    data_raw : pandas.DataFrame
        DataFrame with raw data
    data_ona : pandas.DataFrame
        DataFrame with ONA processed data
    data_cma : pandas.DataFrame
        DataFrame with CMA processed data
    data_dema : pandas.DataFrame
        DataFrame with DEMA processed data
    wavelength : str
        Which wavelength to analyze
    """
    # Identify columns
    bc_col = f"{wavelength} BC1"
    ona_col = f"{wavelength}_BC_ONA"
    cma_col = f"{wavelength}_BC_CMA"
    dema_col = f"{wavelength}_BC_DEMA"
    
    # Check which methods are available
    methods = []
    if bc_col in data_raw.columns:
        raw_data = data_raw[bc_col].copy()
        if ona_col in data_ona.columns:
            methods.append(('ONA', data_ona[ona_col]))
        if cma_col in data_cma.columns:
            methods.append(('CMA', data_cma[cma_col]))
        if dema_col in data_dema.columns:
            methods.append(('DEMA', data_dema[dema_col]))
    else:
        print(f"Raw data for {wavelength} wavelength not available")
        return
    
    if len(methods) == 0:
        print("No processed data available for analysis")
        return
    
    # Calculate cross-correlation and find peak for each method
    results = {}
    lags = {}
    peak_corrs = {}
    
    max_lag = 20  # Maximum lag to consider (in data points)
    
    for name, processed_data in methods:
        # Calculate cross-correlation
        # First, standardize the data
        raw_std = (raw_data - raw_data.mean()) / raw_data.std()
        processed_std = (processed_data - processed_data.mean()) / processed_data.std()
        
        # Calculate cross-correlation with different lags
        xcorr = np.correlate(raw_std.fillna(0), processed_std.fillna(0), mode='full')
        
        # Calculate the midpoint
        mid = len(xcorr) // 2
        
        # Extract the central portion of the cross-correlation
        lag_range = max_lag * 2 + 1
        central_xcorr = xcorr[mid - max_lag:mid + max_lag + 1]
        lags_array = np.arange(-max_lag, max_lag + 1)
        
        # Find the peak correlation and its lag
        peak_idx = np.argmax(central_xcorr)
        peak_lag = lags_array[peak_idx]
        peak_corr = central_xcorr[peak_idx]
        
        # Store results
        results[name] = {
            'xcorr': central_xcorr,
            'lags': lags_array
        }
        lags[name] = peak_lag
        peak_corrs[name] = peak_corr
    
    # Plot cross-correlation for each method
    plt.figure(figsize=(10, 6))
    
    for name, result in results.items():
        if name == 'ONA':
            plt.plot(result['lags'], result['xcorr'], 'b-', label=f'ONA (lag={lags[name]})')
        elif name == 'CMA':
            plt.plot(result['lags'], result['xcorr'], 'r-', label=f'CMA (lag={lags[name]})')
        elif name == 'DEMA':
            plt.plot(result['lags'], result['xcorr'], 'g-', label=f'DEMA (lag={lags[name]})')
    
    plt.axvline(x=0, color='k', linestyle='--', alpha=0.5)
    plt.xlabel('Lag (data points)')
    plt.ylabel('Cross-Correlation')
    plt.title(f'Cross-Correlation with Raw Data - {wavelength} Wavelength')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Display results in a table
    summary = {}
    for name in results.keys():
        summary[name] = {
            'Lag (data points)': lags[name],
            'Peak Correlation': peak_corrs[name]
        }
    
    # Convert to DataFrame for display
    summary_df = pd.DataFrame.from_dict(summary, orient='index')
    print(f"\nSignal preservation metrics for {wavelength} wavelength:")
    display(summary_df)

In [ ]:
# Analyze signal preservation for each wavelength
for wavelength in ['Blue', 'IR']:
    print(f"\nSignal preservation analysis for {wavelength} wavelength:")
    analyze_signal_preservation(data, processed_data_ona, processed_data_cma, processed_data_dema, wavelength)


## 15. Comparative Analysis Summary
 
Now let's summarize the performance of all three methods across key metrics.

In [ ]:
def create_method_comparison_table():
    """
    Create a comprehensive comparison table for all three methods
    """
    # Gather all metrics for comparison
    comparison = {
        'Metric': [
            'Negative Value Reduction', 
            'Noise Reduction Factor', 
            'Correlation with Raw Data',
            'Signal Preservation (Lag)',
            'Time Averaging Behavior',
            'Source Apportionment Stability',
            'Computational Complexity'
        ]
    }
    
    # Add columns for each method
    methods = ['ONA', 'CMA', 'DEMA']
    for method in methods:
        comparison[method] = [
            # To be filled with actual values where available
            'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A'
        ]
    
    # Create DataFrame
    comparison_df = pd.DataFrame(comparison)
    comparison_df.set_index('Metric', inplace=True)
    
    # Fill in values from summary_table where available
    if 'summary_table' in globals():
        # Negative Value Reduction
        for method in methods:
            for wavelength in ['Blue', 'IR']:
                key = f"{wavelength}_{method}"
                if key in summary_table.index:
                    if comparison_df.loc['Negative Value Reduction', method] == 'N/A':
                        comparison_df.loc['Negative Value Reduction', method] = f"{summary_table.loc[key, 'Negative reduction']:.2f}"
                    else:
                        current = float(comparison_df.loc['Negative Value Reduction', method])
                        new_value = (current + summary_table.loc[key, 'Negative reduction']) / 2
                        comparison_df.loc['Negative Value Reduction', method] = f"{new_value:.2f}"
                    
                    # Noise Reduction
                    if comparison_df.loc['Noise Reduction Factor', method] == 'N/A':
                        comparison_df.loc['Noise Reduction Factor', method] = f"{summary_table.loc[key, 'Noise reduction factor']:.1f}x"
                    else:
                        current = float(comparison_df.loc['Noise Reduction Factor', method].replace('x', ''))
                        new_value = (current + summary_table.loc[key, 'Noise reduction factor']) / 2
                        comparison_df.loc['Noise Reduction Factor', method] = f"{new_value:.1f}x"
                    
                    # Correlation
                    if comparison_df.loc['Correlation with Raw Data', method] == 'N/A':
                        comparison_df.loc['Correlation with Raw Data', method] = f"{summary_table.loc[key, 'Correlation with raw']:.3f}"
                    else:
                        current = float(comparison_df.loc['Correlation with Raw Data', method])
                        new_value = (current + summary_table.loc[key, 'Correlation with raw']) / 2
                        comparison_df.loc['Correlation with Raw Data', method] = f"{new_value:.3f}"
    
    # Fill in qualitative assessments 
    # Signal Preservation (Lag)
    comparison_df.loc['Signal Preservation (Lag)', 'ONA'] = "No lag"
    comparison_df.loc['Signal Preservation (Lag)', 'CMA'] = "Minimal lag"
    comparison_df.loc['Signal Preservation (Lag)', 'DEMA'] = "Some lag possible"
    
    # Time Averaging Behavior
    comparison_df.loc['Time Averaging Behavior', 'ONA'] = "Adaptive based on ΔATN"
    comparison_df.loc['Time Averaging Behavior', 'CMA'] = "Fixed window"
    comparison_df.loc['Time Averaging Behavior', 'DEMA'] = "Exponential weighting"
    
    # Source Apportionment Stability
    comparison_df.loc['Source Apportionment Stability', 'ONA'] = "Good"
    comparison_df.loc['Source Apportionment Stability', 'CMA'] = "Better"
    comparison_df.loc['Source Apportionment Stability', 'DEMA'] = "Best"
    
    # Computational Complexity
    comparison_df.loc['Computational Complexity', 'ONA'] = "High (adaptive)"
    comparison_df.loc['Computational Complexity', 'CMA'] = "Low (simple)"
    comparison_df.loc['Computational Complexity', 'DEMA'] = "Medium"
    
    return comparison_df

## 16. Conclusions and Method Recommendations
 
This notebook implemented and compared three post-processing algorithms for aethalometer data: ONA, CMA, and DEMA.

In [ ]:
# Create and display the overall comparison table
print("\n===== Overall Method Comparison =====")
comparison_table = create_method_comparison_table()
display(comparison_table)